Giving access to the drive

In [6]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Installing detectron2

In [ ]:
!pip install pyyaml==5.1

import torch
TORCH_VERSION = ".".join(torch.__version__.split(".")[:2])
CUDA_VERSION = torch.__version__.split("+")[-1]
print("torch: ", TORCH_VERSION, "; cuda: ", CUDA_VERSION)
# Install detectron2 that matches the above pytorch version
# See https://detectron2.readthedocs.io/tutorials/install.html for instructions
!pip install detectron2 -f https://dl.fbaipublicfiles.com/detectron2/wheels/$CUDA_VERSION/torch$TORCH_VERSION/index.html
# If there is not yet a detectron2 release that matches the given torch + CUDA version, you need to install a different pytorch.

# exit(0)  # After installation, you may need to "restart runtime" in Colab. This line can also restart runtime

In [ ]:
# Some basic setup:
# Setup detectron2 logger
import detectron2
from detectron2.utils.logger import setup_logger
setup_logger()

# import some common libraries
import numpy as np
import os, json, cv2, random
from google.colab.patches import cv2_imshow

#Drive access
from google.colab import drive
drive.mount('/content/gdrive')

# import some common detectron2 utilities
from detectron2 import model_zoo
from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg
from detectron2.utils.visualizer import Visualizer
from detectron2.data import MetadataCatalog, DatasetCatalog

**Registering the datasets** : Here Resgestering the dataset beause that datset is custom format.

In [2]:
import os
import numpy as np
import json
from detectron2.structures import BoxMode

def get_amazonrobo_dicts(directory):
    classes = ['blueball', 'greenball', 'orangeblock', 'cube', 'greenbox', 'woodenblock', 'whitebox', 'blackbag', 'ereaser', 'battery'] #10 classes
    dataset_dicts = []
    for filename in [file for file in os.listdir(directory) if file.endswith('.json')]:
        json_file = os.path.join(directory, filename)
        with open(json_file) as f:
            img_anns = json.load(f)

        record = {}
        
        filename = os.path.join(directory, img_anns["imagePath"])
        
        record["file_name"] = filename
        record["height"] = 720
        record["width"] = 1280
      
        annos = img_anns["shapes"]
        objs = []
        for anno in annos:
            px = [a[0] for a in anno['points']]
            py = [a[1] for a in anno['points']]
            poly = [(x, y) for x, y in zip(px, py)]
            poly = [p for x in poly for p in x]

            obj = {
                "bbox": [np.min(px), np.min(py), np.max(px), np.max(py)],
                "bbox_mode": BoxMode.XYXY_ABS,
                "segmentation": [poly],
                "category_id": classes.index(anno['label']),
                "iscrowd": 0
            }
            objs.append(obj)
        record["annotations"] = objs
        dataset_dicts.append(record)
    return dataset_dicts
#inside train and val: .png and .json
from detectron2.data import DatasetCatalog, MetadataCatalog
for d in ["train", "valid"]:
    DatasetCatalog.register("my_dataset_" + d, lambda d=d: get_amazonrobo_dicts('/content/gdrive/MyDrive/NN_class10/'+ d)) #path to train and val directory in the drive
    MetadataCatalog.get("my_dataset_" + d).set(thing_classes=['blueball', 'greenball', 'orangeblock', 'cube', 'greenbox', 'woodenblock', 'whitebox', 'blackbag', 'ereaser', 'battery'])
amazonrobo_metadata = MetadataCatalog.get("my_dataset_train")

**Verifying the data loading:** Randomly picking 4 samples to visualize

In [ ]:
dataset_dicts = get_amazonrobo_dicts("/content/gdrive/MyDrive/NN_class10/train")
for d in random.sample(dataset_dicts, 2):  #checking 2 random sample
    img = cv2.imread(d["file_name"])
    visualizer = Visualizer(img[:, :, ::-1], metadata=amazonrobo_metadata, scale=0.5)
    out = visualizer.draw_dataset_dict(d)
    cv2_imshow(out.get_image()[:, :, ::-1])

**Training the model** 

In [ ]:
from detectron2.engine import DefaultTrainer
from detectron2.config import get_cfg

cfg = get_cfg()
#Use either RetinaNet or Mask R-CNN
#model_file = "COCO-Detection/retinanet_R_50_FPN_3x.yaml" # RetinaNet for bounding box detection
model_file = "COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml" # Mask R-CNN for instance segmentation

cfg.merge_from_file(model_zoo.get_config_file(model_file))
cfg.DATASETS.TRAIN = ("my_dataset_train",)
cfg.DATASETS.TEST = ()
cfg.DATALOADER.NUM_WORKERS = 2
cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url(model_file)
cfg.SOLVER.IMS_PER_BATCH = 2     # Lower to reduce memory usage (1 is the lowest)
cfg.SOLVER.BASE_LR = 0.00025      # Base learning rate
cfg.SOLVER.MAX_ITER = 1800        # Maximum number of iterations
cfg.MODEL.ROI_HEADS.NUM_CLASSES = 10    #number of classes  
os.makedirs(cfg.OUTPUT_DIR, exist_ok=True)
# Create a default training pipeline and begin training
trainer = DefaultTrainer(cfg) 
trainer.resume_or_load(resume=False)
trainer.train()
#cfg.TEST.EVAL_PERIOD = 100

**Training curve in tensorboard**

In [ ]:
# Looking at training curves in tensorboard:
%load_ext tensorboard
%tensorboard --logdir output

**Validating :** Now runnning the trained model on validating dataset

In [5]:
cfg.MODEL.WEIGHTS = os.path.join(cfg.OUTPUT_DIR, "model_final.pth") # path to the model we just trained
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.7        # set a custom testing threshold
cfg.DATASETS.TEST = ("my_dataset_valid", )
predictor = DefaultPredictor(cfg)

In [ ]:
from detectron2.utils.visualizer import ColorMode
dataset_dicts = get_amazonrobo_dicts('/content/gdrive/MyDrive/NN_class10/val')
#Validating 4 images from validating dataset
for i in range(4):    
    im = cv2.imread(d["file_name"])
    outputs = predictor(im)
    v = Visualizer(im[:, :, ::-1],
                   metadata=amazonrobo_metadata, 
                   scale=0.5, 
                   instance_mode=ColorMode.IMAGE_BW   # remove the colors of unsegmented pixels
    )
    out = v.draw_instance_predictions(outputs["instances"].to("cpu"))
    cv2_imshow(out.get_image()[:, :, ::-1])

## Running Detectron2 on test dataset

In [15]:
import pickle
filename = '/content/gdrive/MyDrive/NN_class10/config.pkl'
with open(filename, 'wb') as f:
     pickle.dump(cfg, f)

#### Generalizing with a single image.

In [ ]:
filename = '/content/gdrive/MyDrive/NN_class10/config.pkl'
with open(filename, 'rb') as f:
     cfg = pickle.load(f)
predictor = DefaultPredictor(cfg)
def GetMask(img):
    outputs = predictor(img)
    v = Visualizer(im[:, :, ::-1], 
                   MetadataCatalog.get(cfg.DATASETS.TRAIN[0]), 
                   scale=0.7, 
                   instance_mode=ColorMode.IMAGE_BW)
    v = v.draw_instance_predictions(outputs["instances"].to("cpu"))
    
    return v.get_image()[:, :, ::-1]

im = cv2.imread("/content/gdrive/MyDrive/NN_class10/test/download (39).png")
cv2_imshow(GetMask(im))